### Initialization

In [ ]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 33.6 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
env = gym.make('CartPole-v1')

### Previous tries

In [ ]:
observation, info = env.reset()

n_fallen = 0
for _ in range(1000):
  # Choose an action

  # polynomial regression pytorch


  # angle -> direction
  # if observation[2] < 0:
  #   action = 0
  # else:
  #   action = 1

  # random choice
  action = env.action_space.sample()  # agent policy that uses the observation and info

  observation, reward, terminated, truncated, info = env.step(action)

  if terminated or truncated:
    observation, info = env.reset()
    n_fallen += 1

env.close()

print(n_fallen)

46


In [ ]:
import torch
import numpy as np
from torch import nn

# Build the model:
class PolynomialRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.p1 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p2 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p3 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p4 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p5 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p6 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.p7 = nn.Parameter(torch.rand( 1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.fc1 = nn.Linear(4, 2)

        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.p1*x**6 + self.p2*x**5 + self.p3*x**4 + self.p4*x**3 + self.p5*x**2 + self.p6*x + self.p7
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [ ]:
observation, info = env.reset()

# Create the model
torch.manual_seed(42)
model = PolynomialRegressionModel()

# Define the loss function and the optimizer
loss_fn = nn.MSELoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(params = model.parameters(),
                            lr = learning_rate)
model.train()

for _ in range(1000):
  # Choose an action
  # polynomial regression pytorch
  y_pred = model(torch.from_numpy(observation))
  action = env.action_space.sample()

  # Loss calculation
  good = True if observation[2] > -0.2 and observation[2] < 0.2 else False
  y_true = np.array([0, 0], dtype='float32')
  y_true[torch.argmax(y_pred) if good else 1 ^ torch.argmax(y_pred)] = 1
  y_true = torch.from_numpy(y_true)
  loss = loss_fn(y_pred, y_true)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  observation, reward, terminated, truncated, info = env.step(action)

  if terminated or truncated:
    observation, info = env.reset()

# AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

n_fallen = 0
for _ in range(1000):
  # Choose an action
  # polynomial regression pytorch
  y_pred = model(torch.from_numpy(observation))
  action = torch.argmax(y_pred).item()

  # Loss calculation
  good = True if observation[2] > -0.2 and observation[2] < 0.2 else False
  y_true = np.array([0, 0], dtype='float32')
  y_true[torch.argmax(y_pred) if good else 1 ^ torch.argmax(y_pred)] = 1
  y_true = torch.from_numpy(y_true)
  loss = loss_fn(y_pred, y_true)

  observation, reward, terminated, truncated, info = env.step(action)

  if terminated or truncated:
    observation, info = env.reset()
    n_fallen += 1

env.close()

print(n_fallen)

101


In [ ]:
torch.manual_seed(42)
model = PolynomialRegressionModel()
model(torch.from_numpy(observation))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([0.3886, 0.6114], grad_fn=<SoftmaxBackward0>)

In [ ]:
print(observation)
print(observation**2)

[-0.00374048  0.22145137  0.01048327 -0.27827266]
[1.3991186e-05 4.9040709e-02 1.0989888e-04 7.7435672e-02]


### Deep Q-Learning

In [ ]:
import numpy as np
import copy
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
class SimpleLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(5, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [ ]:
def tensor_cat(a_np, b_int):
  return torch.concatenate((torch.from_numpy(a_np), torch.tensor([b_int])))

#### Training

In [ ]:
num_iterations = 1000
replay_buffer = []
p_random_choice = 0.2 # probability to do exploration
model = SimpleLinear()
prev_model = copy.deepcopy(model)
optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.001)
num_minibatch = 50
future_importance = 1 # TODO: NOT SURE WHAT VALUE THIS CONSTANT SHOULD HAVE
model_update_steps = 50 # TODO: NOT SURE WHAT VALUE THIS CONSTANT SHOULD HAVE

observation, info = env.reset()

with torch.no_grad():
  for iteration in tqdm(range(num_iterations)):
    # Sampling phase
    if np.random.rand() < p_random_choice:
      action = env.action_space.sample()
    else:
      action = np.argmax([model(tensor_cat(observation, action_variant)) for action_variant in [0, 1]])
    prev_observation = observation
    observation, reward, terminated, truncated, info = env.step(action)
    replay_buffer.append((prev_observation, action, reward, observation, terminated or truncated))

    # Training phase
    if len(replay_buffer) < num_minibatch:
      continue

    replay_indices = np.random.choice(len(replay_buffer), min(num_minibatch, len(replay_buffer)))
    for i_replay in replay_indices:
      sample = replay_buffer[i_replay]
      if sample[4]: # If last transition
        y_true = sample[2]
      else:
        # The error was here.
        # I calculated q_value_true using initial observation from sample,
        # but it should have been the next observation, so it will connect future rewards
        # and proagate them back to this state from the very end.
        q_value_true = max([prev_model(tensor_cat(sample[3], action_variant)) for action_variant in [0, 1]])
        y_true = sample[2] + future_importance * q_value_true
      # Gradient descent step
      with torch.enable_grad():
        loss = (y_true - model(tensor_cat(sample[0], sample[1]))) ** 2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if iteration % model_update_steps == model_update_steps - 1:
      prev_model = copy.deepcopy(model)

    # Reset
    if terminated or truncated:
      observation, info = env.reset()

env.close()

100%|██████████| 1000/1000 [00:44<00:00, 22.41it/s]


#### Testing in 1000 interations

In [ ]:
observation, info = env.reset()

n_fallen = 0
with torch.no_grad():
  for iteration in tqdm(range(1000)):
    action = np.argmax([model(tensor_cat(observation, action_variant)) for action_variant in [0, 1]])
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
      observation, info = env.reset()
      n_fallen += 1

env.close()

print()
print(n_fallen)

100%|██████████| 1000/1000 [00:00<00:00, 1561.18it/s]


5


### Results

#### Results

| Method | Number of falls |
| --- | --- |
| max falls | ~114 |
| random choice | ~45 |
| angle → direction | ~24 |
| polynomial regression v1 | ~100 |
| deep q-learning (2 improvements) | 6 |

WHYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY CAN'T I JUST WRITE A NORMAL LEARNING ALGORITHM  
polynomial regression v1: by state observation it predicts action  
deep q-learning (2 improvements): sample deep learning algorithm, https://huggingface.co/learn/deep-rl-course/unit3/deep-q-algorithm